In [ ]:
import psycopg2
import os

In [ ]:
data_path = '/media/arkane/database_space/raw_data/reddit/csv_converted/'
filenames = os.listdir(data_path)

filenames_no_ext = [file.replace('.csv', '') for file in filenames]
filenames_no_ext.sort()

file_paths = [data_path + file for file in filenames]
file_paths.sort()

In [ ]:
commnand = ""
def create_tables(name):
    command = f"""
       CREATE TABLE "{name}" (
        id varchar(20) PRIMARY KEY,
        author varchar(50),
        subreddit varchar(50),
        subreddit_type varchar(10),
        body text,
        score int,
        controversiality int,
        created_utc int,
        author_created_utc varchar(20)
    );
    """
    return command

copy_cmd = ""  
def copy_into_table(name, path):
    copy_cmd = f"""
        COPY "{name}" (id, author, subreddit,
            subreddit_type, body, controversiality,
                created_utc, score, author_created_utc)
        FROM '{path}'
        DELIMITER ','
        NULL ''
        ENCODING 'UTF8'
        CSV HEADER;
    """
    return copy_cmd

In [ ]:
try:
    
    # Create psql connection
    conn = psycopg2.connect("dbname='reddit_comments' host='localhost' user='admin' password='theLegend30$$'")
    cur = conn.cursor()
    
    
    for fn, pt in zip(filenames_no_ext, file_paths):
        # Create single table using filename
        q1 = create_tables(fn)
        # print(q1)
        cur.execute(q1)
        
        # After try to copy csv into new table
        q2 = copy_into_table(fn, pt)
        # print(q2)
        cur.execute(q2)
        
        # Commit changes
        conn.commit()
        
        if("COPY" in cur.statusmessage):
            os.remove(pt)
            print(f"File '{pt}' was successfully deleted")
    
    # q1 = create_tables(filenames_no_ext[0])
    # cur.execute(q1)
    
    # q2 = copy_into_table(filenames_no_ext[0], file_paths[0])
    # cur.execute(q2)
    
    # conn.commit()
    
    # if("COPY" in cur.statusmessage):
    #     os.remove(file_paths[0])
    #     print(f"File '{file_paths[0]}' was successfully deleted")

except(Exception, psycopg2.DatabaseError) as error:
    conn.rollback()
    print(error)

finally:
    if conn is not None:
        cur.close()
        conn.close()
    